In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Google A2A (Agent-to-Agent) Communication

This notebook introduces you to Google's Agent-to-Agent (A2A) protocol, a standardized way for AI agents to communicate and collaborate.  

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynb">
      <img src="https://avatars.githubusercontent.com/u/33467679?s=200&v=4" width="32px" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynbb">
      <img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/quickstart/agent2agent/a2a_quickstart.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fai-ml-recipes%2Fmain%2Fnotebooks%2Fquickstart%2Fagent2agent%2Fa2a_quickstart.ipynb">
    <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
    Open in Colab Enterprise
    </a>
  </td>

</table>

## What You'll Build

A three-agent system that works together to analyze trending topics:
1. **Trending Topics Agent** - Searches the web for current trending topics
2. **Trend Analyzer Agent** - Performs deep analysis with quantitative data
3. **Host Agent** - Orchestrates the other agents to provide comprehensive insights

<img src="../../../docs/images/a2a-diagram.png" alt="drawing" width="1000"/>


## Prerequisites

- Python 3.11+
- Google Cloud Project with Vertex AI enabled
- Basic understanding of async Python

## Other Resources

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol Specification](https://github.com/google/a2a)
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai)
- Codelabs:
  - [Google's Agent Stack in Action: ADK, A2A, MCP on Google Cloud](https://codelabs.developers.google.com/instavibe-adk-multi-agents/instructions)
  - [Getting Started with Agent-to-Agent (A2A) Protocol: Gemini on Cloud Run](https://codelabs.developers.google.com/intro-a2a-purchasing-concierge)
  - [Getting Started with MCP, ADK and A2A](https://codelabs.developers.google.com/codelabs/currency-agent)

#### Important!
A2A is a work in progress (WIP) thus, in the near future there might be changes that are different from what demonstrated in this code.

### Setup and Installation

First, let's install the required dependencies:

In [ ]:
# Install required packages
!pip3 install --upgrade -q google-genai google-adk>=1.6.1 a2a-sdk python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

## 1. Introduction to A2A

### What is Agent-to-Agent (A2A) Communication?

A2A is a standardized protocol that enables AI agents to:
- **Discover** each other's capabilities
- **Communicate** using a common JSON-RPC based protocol
- **Collaborate** to solve complex tasks
- **Stream** responses for real-time interactions

### Architecture Overview

### Environment Configuration

In [ ]:
import os
import sys

# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'dataproc-workspaces-notebooks'  # Replace with your project ID
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'  # Replace with your location

from dotenv import load_dotenv
load_dotenv()

print("Environment variables configured:")
print(f"GOOGLE_GENAI_USE_VERTEXAI: {os.environ['GOOGLE_GENAI_USE_VERTEXAI']}")
print(f"GOOGLE_CLOUD_PROJECT: {os.environ['GOOGLE_CLOUD_PROJECT']}")
print(f"GOOGLE_CLOUD_LOCATION: {os.environ['GOOGLE_CLOUD_LOCATION']}")

In [ ]:
# Authenticate your notebook environment (Colab only)
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

## 2. Building Your A2A System

Let's build our three-agent system step by step. We'll create:

1. **Trending Topics Agent** - Finds current trending topics
2. **Trend Analyzer Agent** - Analyzes trends with quantitative data
3. **Host Agent** - Orchestrates the other agents

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService

from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutorConfig

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore

from a2a.types import AgentCard, AgentCapabilities, AgentSkill

### Agent 1: Trending Topics Agent

This agent searches the web for trending topics and returns a list of current trends.

In [ ]:
# Create the Trending Topics ADK Agent
trending_agent = Agent(
    model="gemini-2.5-pro",
    name="trending_topics_agent",
    instruction="""
    You are a social media trends analyst. Your job is to search the web for current trending topics,
    particularly from social platforms.
    
    When asked about trends:
    1. Search for "trending topics today" or similar queries
    2. Extract the top 3 trending topics
    3. Return them in a JSON format
    
    Focus on current, real-time trends from the last 24 hours.
    
    You MUST return your response in the following JSON format:
    {
        "trends": [
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)", 
                "reason": "Why it's trending"
            },
            {
                "topic": "Topic name",
                "description": "Brief description (1-2 sentences)",
                "reason": "Why it's trending"
            }
        ]
    }
    
    Only return the JSON object, no additional text.
    """,
    tools=[google_search],
)

print("Trending Topics Agent created successfully!")

In [ ]:
trending_agent_card = AgentCard(
      name="Trending Topics Agent",
      url="http://localhost:10020",
      description="Searches the web for current trending topics from social media",
      version="1.0",
      capabilities=AgentCapabilities(streaming=True),
      defaultInputModes=["text/plain"],
      defaultOutputModes=["application/json"],
      skills=[
            AgentSkill(
                id="find_trends",
                name="Find Trending Topics",
                description="Searches for current trending topics on social media",
                tags=["trends", "social media", "twitter", "current events"],
                examples=[
                    "What's trending today?",
                    "Show me current Twitter trends",
                    "What are people talking about on social media?",
                ],
            )
      ],
  )

In [ ]:
remote_trending_agent = RemoteA2aAgent(
    name="find_trends",
    description="Searches for current trending topics on social media",
    agent_card="http://localhost:10020/.well-known/agent.json"
)

### Agent 2: Trend Analyzer Agent

This agent takes a specific trend and performs deep analysis with quantitative data.

In [ ]:
# Create the Trend Analyzer ADK Agent
analyzer_agent = Agent(
    model="gemini-2.5-pro",
    name="trend_analyzer_agent",
    instruction="""
    You are a data analyst specializing in trend analysis. When given a trending topic,
    perform deep research to find quantitative data and insights.
    
    For each trend you analyze:
    1. Search for statistics, numbers, and metrics related to the trend
    2. Look for:
       - Engagement metrics (views, shares, mentions)
       - Growth rates and timeline
       - Geographic distribution
       - Related hashtags or keywords
    3. Provide concrete numbers and data points

    Keep it somehow concise
    
    Always prioritize quantitative information over qualitative descriptions.
    """,
    tools=[google_search],
)

print("Trend Analyzer Agent created successfully!")

In [ ]:
analyzer_agent_card = AgentCard(
      name="Trend Analyzer Agent",
      url="http://localhost:10021",
      description="Performs deep analysis of trends with quantitative data",
      version="1.0",
      capabilities=AgentCapabilities(streaming=True),
      defaultInputModes=["text/plain"],
      defaultOutputModes=["application/json"],
      skills=[
            AgentSkill(
                id="analyze_trend",
                name="Analyze Trend",
                description="Provides quantitative analysis of a specific trend",
                tags=["analysis", "data", "metrics", "statistics"],
                examples=[
                    "Analyze the #ClimateChange trend",
                    "Get metrics for the Taylor Swift trend",
                    "Provide data analysis for AI adoption trend",
                ],
            )
      ],
  )

In [ ]:
remote_analyzer_agent = RemoteA2aAgent(
    name="analyze_trend",
    description="Provides quantitative analysis of a specific trend",
    agent_card="http://localhost:10021/.well-known/agent.json"
)

### Agent 3: Host Agent (Orchestrator)

The Host Agent coordinates between the other two agents to provide comprehensive trend analysis.

In [ ]:
# Create the Host ADK Agent
host_agent = Agent(
    model="gemini-2.5-pro",
    name="trend_analysis_host",
    instruction="""
You are an expert AI Orchestrator.
Your primary responsibility is to intelligently interpret user requests, plan the necessary sequence of actions if multiple steps are involved, and delegate them to the most appropriate specialized remote agents.
You do not perform the tasks yourself but manage their assignment, sequence, and can monitor their status.

Core Workflow & Decision Making:

1.  **Understand User Intent & Complexity:**
    *   Carefully analyze the user's request to determine the core task(s) they want to achieve. Pay close attention to keywords and the overall goal.
    *   **Identify if the request requires a single agent or a sequence of actions from multiple agents.** For example, it could require two agents to be called in sequence.

2.  **Agent Discovery & Selection:**
    *   You have access to sub_agents with specific capabilities (e.g., what kind of requests each agent is designed to handle and what data they output).
    *   Based on the user's intent:
        *   For **single-step requests**, select the single most appropriate agent.
        *   For **multi-step requests**, identify all necessary agents and determine the logical order of their execution.

3.  **Task Planning & Sequencing (for Multi-Step Requests):**
    *   Before delegating, outline the sequence of agent tasks.
    *   Identify dependencies: Does Agent B need information from Agent A's completed task?
    *   Plan to execute tasks sequentially if there are dependencies, waiting for the completion of a prerequisite task before initiating the next one.

4.  **Task Delegation & Management:**
    *   **For New Single Requests or the First Step in a Sequence:**:
        *   The `message` extracted from the user's input, formatted in a way the target agent will understand (check the agent info to better structure the message). 
    *   **For Subsequent Steps in a Sequence:**
        *   Once the prerequisite task is done, gather any necessary output from it.
        *   Then, use the next agent in the sequence, providing it with the user's original relevant intent and any necessary data obtained from the previous agent's task.

**Communication with User:**

*   Clearly inform the user which agent is handling each task. The user should know the entire sequence of agents you used and the results of each one.
*   If the user's request is ambiguous, if necessary information is missing for any agent in the sequence, or if you are unsure about the plan, proactively ask the user for clarification.
*   Rely strictly on your tools and the information they provide.
*   Communicate to the user the content from the data gathered from the all remote agents responses.
*   The communication to the user should contain most of the information from the remote agents, do not summarize too much.

**Important Reminders:**
*   Always prioritize selecting the correct agent(s) based on their documented purpose.
*   Ensure all information required by the chosen remote agent is included in the call, including outputs from previous agents if it's a sequential task.
*   Focus on the most recent parts of the conversation for immediate context, but maintain awareness of the overall goal, especially for multi-step requests.

""",
    sub_agents=[remote_trending_agent, remote_analyzer_agent]
)

In [ ]:
host_agent_card = AgentCard(
    name="Trend Analysis Host",
    url="http://localhost:10022",
    description="Orchestrates trend discovery and analysis using specialized agents",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    defaultInputModes=["text/plain"],
    defaultOutputModes=["application/json"],
    skills=[
        AgentSkill(
            id="comprehensive_trend_analysis",
            name="Comprehensive Trend Analysis",
            description="Finds trending topics and provides deep analysis of the most relevant one",
            tags=["trends", "analysis", "orchestration", "insights"],
            examples=[
                "Analyze current trends",
                "What's trending and why is it important?",
                "Give me a comprehensive trend report",
            ],
        )
    ],
)

## 3. Running

Now let's put everything together. We'll create helper functions to start our agents and run the complete system.

### Starting the A2A Servers

Create function to run each remote agent as an A2A server:

In [ ]:
def create_agent_a2a_server(
    agent, 
    agent_card
):
    """
    Create an A2A server for any ADK agent.
    
    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card
    
    Returns:
        A2AStarletteApplication instance
    """

    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(
        runner=runner, config=config
    )
    
    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler
    )

In [ ]:
import threading
import asyncio
import time
import nest_asyncio
import uvicorn

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Global servers to keep references
servers = []

async def run_server_notebook(create_agent_function, port):
    """Run server with proper error handling."""
    try:
        print(f"🚀 Starting agent on port {port}...")
        app = create_agent_function()
        config = uvicorn.Config(
            app.build(), 
            host="127.0.0.1",  
            port=port, 
            log_level="error",  
            loop="asyncio"
        )
        server = uvicorn.Server(config)
        servers.append(server)
        await server.serve()
    except Exception as e:
        print(f"Agent error: {e}")

def run_agent_in_background(create_agent_function, port, name):
    """Run an agent server in a background thread."""
    def run():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        try:
            # Create the coroutine inside the new event loop
            loop.run_until_complete(run_server_notebook(create_agent_function, port))
        except Exception as e:
            print(f"{name} error: {e}")
    
    thread = threading.Thread(target=run, daemon=True)
    thread.start()
    return thread

# Start agent servers with corrected function calls
print("Starting agent servers...\n")

trending_thread = run_agent_in_background(lambda: create_agent_a2a_server(trending_agent, trending_agent_card), 10020, "Trending Agent")
analyzer_thread = run_agent_in_background(lambda: create_agent_a2a_server(analyzer_agent, analyzer_agent_card), 10021, "Analyzer Agent")
host_thread = run_agent_in_background(lambda: create_agent_a2a_server(host_agent, host_agent_card), 10022, "Host Agent")

# Wait for servers to start
time.sleep(3)

# Check if threads are alive
if trending_thread.is_alive() and analyzer_thread.is_alive():
    print("\n✅ Agent servers are running!")
    print("   - Trending Agent: http://127.0.0.1:10020")
    print("   - Analyzer Agent: http://127.0.0.1:10021")
    print("   - Host Agent: http://127.0.0.1:10022")
else:
    print("\n❌ Agent servers failed to start. Check the error messages above.")

## 4. Testing the System

### Call the A2A agents (the 2 remote agents, and the host agent that refers to the 2 remote agents as sub agents)

In [ ]:
import json
import uuid
import httpx
from typing import Any

from a2a.client import A2AClient
from a2a.types import (
    AgentCard,
    SendMessageRequest,
    MessageSendParams,
)

class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[str, dict[str, Any] | None] = {} # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if agent_url in self._agent_info_cache and self._agent_info_cache[agent_url] is not None:
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(f"{agent_url}/.well-known/agent.json")
                agent_card_data = self._agent_info_cache[agent_url] = agent_card_response.json()

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            client = A2AClient(
                httpx_client=httpx_client,
                agent_card=agent_card
            )

            # Build the message parameters following official structure
            send_message_payload = {
                'message': {
                    'role': 'user',
                    'parts': [
                        {'kind': 'text', 'text': message}
                    ],
                    'messageId': uuid.uuid4().hex,
                }
            }

            # Create the request
            request = SendMessageRequest(
                id=str(uuid.uuid4()),
                params=MessageSendParams(**send_message_payload)
            )

            # Send the message with timeout configuration
            response = await client.send_message(request)

            # Extract text from response
            try:
                response_dict = response.model_dump(mode='json', exclude_none=True)
                if 'result' in response_dict and 'artifacts' in response_dict['result']:
                    artifacts = response_dict['result']['artifacts']
                    for artifact in artifacts:
                        if 'parts' in artifact:
                            for part in artifact['parts']:
                                if 'text' in part:
                                    return part['text']

                # If we couldn't extract text, return the full response as formatted JSON
                return json.dumps(response_dict, indent=2)

            except Exception as e:
                # Log the error and return string representation
                print(f"Error parsing response: {e}")
                return str(response)

In [ ]:
a2a_client = A2ASimpleClient()

In [ ]:
trending_topics = await a2a_client.create_task("http://localhost:10020", "What's trending today?")
print(trending_topics)
     

In [ ]:
analysis = await a2a_client.create_task("http://localhost:10021", "Analyze the trend AI in Social Media")
print(analysis)
     

In [ ]:
host_analysis = await a2a_client.create_task("http://localhost:10022", "Find the most relevant trends in the web today, choose randomly one of the top trends, and give me a complete analysis of it with quantitative data")
print(host_analysis)

## Summary

Congratulations! You've successfully built a multi-agent system using Google's A2A protocol. Here's what you've learned:

1. **A2A Protocol Basics**: How agents discover and communicate with each other
2. **ADK Integration**: Creating ADK agents and wrapping them for A2A
3. **Agent Orchestration**: Building a Host Agent that coordinates multiple agents
4. **Practical Implementation**: Running and testing a complete multi-agent system

### Next Steps

- **Deploy Your Agents**: Deploy agents to Cloud Run or other platforms
- **Add Authentication**: Implement security for production use
- **Create More Agents**: Build agents for your specific use cases, even using other frameworks
- **Advanced Patterns**: Explore agent chains, parallel execution, and more
- **Callbacks**: Add in the Google ADK agents the before and after callbacks of the agent, model and tool, to increase observability

Happy agent building! 🚀

# Appendix

### Why Use Google A2A (Agent-to-Agent) Protocol

Google's Agent-to-Agent (A2A) protocol is a standardized communication framework that enables AI agents to discover, communicate, and collaborate with each other using a common JSON-RPC based protocol.  
It provides a uniform way for agents to interact, regardless of their underlying implementation.  

#### 1. Standardized Communication Protocol

- A2A provides a consistent, JSON-RPC based protocol that any agent can implement
- Agents can communicate without needing to know each other's internal implementation details
- The protocol supports streaming responses for real-time interactions

#### 2. Agent Discovery and Metadata

- Agents expose their capabilities through standardized metadata (AgentCard)
- Each agent publishes its skills, input/output modes, and capabilities
- Host agents can dynamically discover what other agents can do through the `.well-known/agent.json` endpoint

#### 3. Orchestration and Composition

- Enables building complex multi-agent systems where a host agent can orchestrate multiple specialized agents
- Supports sequential and parallel task execution patterns
- Allows for sophisticated agent collaboration workflows

#### 4. Platform Independence

- A2A servers can wrap agents from different frameworks (not just ADK)
- Agents can be deployed as independent services on different infrastructure
- Promotes loose coupling between agents

### Differences: Using ADK Agents Directly vs. Through A2A

#### Using ADK Agents Directly

```python
# Conceptual Example: Defining Hierarchy
from google.adk.agents import LlmAgent, BaseAgent

# Define individual agents
greeter = LlmAgent(name="Greeter", model="gemini-2.5-pro")
task_doer = BaseAgent(name="TaskExecutor") # Custom non-LLM agent

# Create parent agent and assign children via sub_agents
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.5-pro",
    description="I coordinate greetings and tasks.",
    sub_agents=[ # Assign sub_agents here
        greeter,
        task_doer
    ]
)
```

__Use Direct ADK for Multi-Agents System When:__

- All agents are tightly related and always used together
- Google ADK is the framework choice, and simplicity is prioritized
- Performance of in-process communication is critical
- You don't need distributed deployment
- No built-in service discovery is needed

#### Using ADK Agents Through A2A

__Use A2A for Multi-Agents System When:__

- Building complex multi-agent systems
- Agents need to be developed, deployed, and scaled independently
- You want to integrate agents from different teams or frameworks
- You need dynamic agent discovery and composition
- Building a platform where agents can be added/removed dynamically
- You want to enable third-party agent integration